In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

#create database
kill_match_db = create_engine('sqlite:///kill_match.db')
df1 = pd.read_sql_query('select killed_by, killer_position_x, killer_position_y, victim_position_x, victim_position_y from kmatch where killed_by not like "%death.%" and killed_by not like "%Down%" and killed_by not like "%Blue%" and killed_by not like "%Fall%" and killed_by like "SCAR_L" limit 10000', kill_match_db)




In [ ]:
df1['kill_dist'] = ((df1['killer_position_x'] - df1['victim_position_x'])**2 + 
                    (df1['killer_position_y'] - df1['victim_position_y'])**2)**(1/2)


In [ ]:
def iterate(k):
    cluster_centers = []
    convergence = 0
    
    #Random choose k centers
    for i in range (k):
        index = np.random.choice(len(df1['kill_dist']))
        cluster_centers.append(round(df1.loc[index,'kill_dist']))
        
    while convergence !=k:
        convergence = 0
        groups = []
        
        #find the closest center for each point
        for i in range(len(df1['kill_dist'])):
            min_dist = abs(df1.loc[i,'kill_dist'] - cluster_centers[0])
            min_ind = 0
            for j in range(1,k):
                temp_dist = abs(df1.loc[i,'kill_dist'] - cluster_centers[j])
                if temp_dist < min_dist:
                    min_dist = temp_dist
                    min_ind = j
            groups.append(min_ind)
        
        #calculate the new center
        for i in range (k):
            dsum = 0
            count = 0
            for j in range(len(df1['kill_dist'])):
                if groups[j] == i:
                    dsum += df1.loc[j,'kill_dist']
                    count += 1
            if count != 0:
                new_center = dsum/count
            if (new_center == cluster_centers[i]):
                convergence += 1
            else:
                cluster_centers[i] = new_center
        print (convergence)
        print (cluster_centers)
                


In [ ]:
iterate(5)